In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_ones_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/central_pixel_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_zeros_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip


In [18]:
import pandas as pd
import numpy as np
import zipfile
import os
import cv2
import tensorflow as tf

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [20]:
with zipfile.ZipFile("../input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [35]:
data = pd.read_csv("training_solutions_rev1.csv")
data["GalaxyID"] = data["GalaxyID"].astype(str) + ".jpg"
data.info

<bound method DataFrame.info of          GalaxyID  Class1.1  Class1.2  Class1.3  Class2.1  Class2.2  Class3.1  \
0      100008.jpg  0.383147  0.616853  0.000000  0.000000  0.616853  0.038452   
1      100023.jpg  0.327001  0.663777  0.009222  0.031178  0.632599  0.467370   
2      100053.jpg  0.765717  0.177352  0.056931  0.000000  0.177352  0.000000   
3      100078.jpg  0.693377  0.238564  0.068059  0.000000  0.238564  0.109493   
4      100090.jpg  0.933839  0.000000  0.066161  0.000000  0.000000  0.000000   
...           ...       ...       ...       ...       ...       ...       ...   
61573  999948.jpg  0.510379  0.489621  0.000000  0.059207  0.430414  0.000000   
61574  999950.jpg  0.901216  0.098784  0.000000  0.000000  0.098784  0.000000   
61575  999958.jpg  0.202841  0.777376  0.019783  0.116962  0.660414  0.067245   
61576  999964.jpg  0.091000  0.909000  0.000000  0.045450  0.863550  0.022452   
61577  999967.jpg  0.767000  0.140000  0.093000  0.000000  0.140000  0.000000

In [34]:
new_data = data[((data["Class1.1"] >= .8) | (data["Class1.2"] >= .8))].reset_index(drop=True)
new_data.drop_duplicates(inplace=True)
new_data.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100090.jpg,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,100134.jpg,0.021834,0.976952,0.001214,0.021751,0.955201,0.313077,0.642124,0.546491,0.408711,...,0.021751,0.207253,0.152044,0.187194,0.000000,0.00000,0.054649,0.081974,0.081974,0.327894
2,100237.jpg,0.164391,0.800759,0.034850,0.030795,0.769964,0.149037,0.620928,0.387742,0.382223,...,0.000000,0.208685,0.179056,0.000000,0.032728,0.14098,0.000000,0.000000,0.089185,0.124848
3,100263.jpg,0.179654,0.818530,0.001816,0.573791,0.244739,0.047326,0.197413,0.016623,0.228116,...,0.141605,0.000000,0.007855,0.008768,0.000000,0.00000,0.000000,0.000000,0.000000,0.016623
4,100322.jpg,0.091987,0.908013,0.000000,0.816806,0.091207,0.000000,0.091207,0.000000,0.091207,...,0.142815,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [23]:
conditions = [new_data["Class1.1"] >= 0.8, new_data["Class1.2"] >= 0.8]
choices = ["elliptical", "spiral"]
new_data["class"] = np.select(conditions, choices, default="Tie")


In [24]:
df_spiral = new_data[new_data["class"] == "spiral"].sample(n=8000, random_state=42)
df_elliptical = new_data[new_data["class"] == "elliptical"].sample(n=8000, random_state=42)


In [25]:
frames = [df_spiral, df_elliptical]
df_galaxies = pd.concat(frames)[["GalaxyID", "class"]]


In [26]:
with zipfile.ZipFile("../input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [27]:

datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)


In [28]:
train_generator = datagen.flow_from_dataframe(
    dataframe=df_galaxies,
    target_size=(224, 224),
    directory="./images_training_rev1",
    batch_size=32,
    x_col="GalaxyID",
    y_col="class",
    color_mode="rgb",
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=42
)

Found 12800 validated image filenames belonging to 2 classes.


In [29]:
valid_generator = datagen.flow_from_dataframe(
    dataframe=df_galaxies,
    target_size=(224, 224),
    directory="./images_training_rev1",
    batch_size=32,
    x_col="GalaxyID",
    y_col="class",
    color_mode="rgb",
    class_mode="categorical",
    subset="validation",
    shuffle=True,
    seed=42
)

Found 3200 validated image filenames belonging to 2 classes.


In [31]:
def add_lora_adapter(base_model, rank=8):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(rank, activation='linear', name="lora_down")(x)
    x = Dense(512, activation='relu', name="lora_up")(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [32]:
base_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_lora_mobilenet = add_lora_adapter(base_mobilenet)
model_lora_mobilenet.fit(train_generator, epochs=10, validation_data=valid_generator)


I0000 00:00:1744910839.755910      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744910850.208744      74 service.cc:148] XLA service 0x7f191c143790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744910850.209573      74 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1744910851.111484      74 cuda_dnn.cc:529] Loaded cuDNN version 90300


  4/400 ━━━━━━━━━━━━━━━━━━━━ 19s 48ms/step - accuracy: 0.6003 - loss: 0.6902 

I0000 00:00:1744910855.638256      74 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


400/400 ━━━━━━━━━━━━━━━━━━━━ 43s 73ms/step - accuracy: 0.8040 - loss: 0.4252 - val_accuracy: 0.8847 - val_loss: 0.2693
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.9162 - loss: 0.2198 - val_accuracy: 0.8319 - val_loss: 0.4027
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 26s 64ms/step - accuracy: 0.9223 - loss: 0.1941 - val_accuracy: 0.8672 - val_loss: 0.3158
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 26s 64ms/step - accuracy: 0.9276 - loss: 0.1863 - val_accuracy: 0.8828 - val_loss: 0.2975
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 27s 66ms/step - accuracy: 0.9288 - loss: 0.1821 - val_accuracy: 0.9300 - val_loss: 0.1878
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 27s 66ms/step - accuracy: 0.9342 - loss: 0.1694 - val_accuracy: 0.8900 - val_loss: 0.2823
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 26s 63ms/step - accuracy: 0.9339 - loss: 0.1617 - val_accuracy: 0.9328 - val_loss: 0.1765
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 26s 65ms/step - accuracy: 0.9345 - loss: 0.1648 - val_accurac

In [33]:
base_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_lora_inception = add_lora_adapter(base_inception)
model_lora_inception.fit(train_generator, epochs=10, validation_data=valid_generator)


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 52s 82ms/step - accuracy: 0.8352 - loss: 0.3681 - val_accuracy: 0.8647 - val_loss: 0.3454
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 27s 66ms/step - accuracy: 0.9369 - loss: 0.1675 - val_accuracy: 0.9050 - val_loss: 0.2541
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 25s 63ms/step - accuracy: 0.9416 - loss: 0.1463 - val_accuracy: 0.9178 - val_loss: 0.2192
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 27s 67ms/step - accuracy: 0.9469 - loss: 0.1360 - val_accuracy: 0.9438 - val_loss: 0.1592
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 27s 66ms/step - accuracy: 0.9486 - loss: 0.1264 - val_accuracy: 0.9513 - val_loss: 0.1451
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 27s 66ms/step - accuracy: 0.9523 - loss: 0.1221 - val_accuracy: 0.9278 - val_loss: 0.2035
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 27s 66ms/step - accuracy: 0.9566 - loss: 0.1139 - val_accuracy: 0.9191 - val_loss: 0.2160
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━